---
Imports
---

In [ ]:
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings('ignore')
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [ ]:
# Import the .csv files and concat them into one dataframe
original_dataframe = pd.concat(map(pd.read_csv, glob.glob('data/*.csv')))

In [ ]:
# Set a working dataframe, so that we don't have to wait 10s it to import again if we want to start fresh
df = original_dataframe

---
EDA - Part 1
---

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Only a very limited amount of suspended projects (drop), canceled projects will be treated as though they failed
df['state'].value_counts()

In [ ]:
# Check for duplicate projects and store them in a table
dups = df.groupby(df.id.tolist()).size().reset_index().rename(columns={0:'count'})
# Sum the final col of that table, and subtract the number of culprits:
dups['count'].sum() - dups.shape[0]

---
Data Cleaning
---

In [ ]:
# Drop features which will not be needed for further analysis
dropped_features = ['blurb', 'currency_symbol', 'backers_count', 'is_backing', 'permissions', 'is_starred', 'source_url',
                    'slug', 'name', 'static_usd_rate', 'profile', 'friends', 'spotlight', 'is_starrable', 'photo', 'pledged', 'usd_type',
                    'fx_rate', 'location', 'creator', 'currency_trailing_code','current_currency', 'created_at', 'urls']
df = df.drop(dropped_features, axis=1)

In [ ]:
# Built array which contains live projects for later use 
array_live = ['live']
live_projects = df.loc[df['state'].isin(array_live)]

# Filter and concat. for target variable
array_notlive = ['successful', 'failed', 'canceled']
finished_projects = df.loc[df['state'].isin(array_notlive)]
finished_projects.replace('canceled','failed', inplace=True)
# Replace successful and failed entries
finished_projects.replace(['successful','failed'],[1,0], inplace=True)

In [ ]:
# Sort dataframe by 'date_changed_at' so that we will keep the entry that was most recently updated
df.sort_values('state_changed_at')
# Remove duplicates
duplicates = df.duplicated(subset='id', keep='last')
df = df[~duplicates]

---
Feature Engineering
---

In [ ]:
#Extract category names from long string in 'category' column
list = []
for i, j in df['category'].iteritems():
    try:
        found = re.search('slug":"(.+?)/', j).group(1)
        list.append(found)
    except AttributeError:
        pass

list_2 = []
for i, j in enumerate(list):
    try:
        found = re.search('(.+?)"', j).group(1)
        list_2.append(found)
    except AttributeError:
        pass

# Add the categories and delete the original cluttered category
df['categories'] = pd.Series(list_2)
df.drop('category', axis=1, inplace=True)

In [ ]:
# Generate new column with readable timeformat
df['launched_at_new'] = pd.to_datetime(df['launched_at'], unit='s')
df['deadline_new'] = pd.to_datetime(df['deadline'], unit='s')
df['state_changed_at_new'] = pd.to_datetime(df['state_changed_at'], unit='s')

In [ ]:
#create new feature 'duration' that displays wheather the project timespan was more or less than 30 days
df_time = df.eval('duration = deadline - launched_at')
df_time['duration'] = ['over' if x > 2592000 else 'under' for x in df_time['duration']]
df_time

In [ ]:
# change dates to weekend(1) or weekday(0)
def change_time(dataframe, column_list):
    for column in column_list:
        #dataframe[column] = pd.to_datetime(dataframe[column], unit='s')
        dataframe[column] = [1 if x >= 6 else 0 for x in pd.to_datetime(dataframe[column], unit='s').dt.weekday]
    return dataframe

In [ ]:
times_lst = ['launched_at', 'deadline', 'state_changed_at']
change_time(df_time, times_lst)

In [ ]:
# Add new column 'time' that displays the time from project launch to project end
df_time.eval('time = state_changed_at_new - launched_at_new', inplace=True)
df_time

In [ ]:
df_time['time'] = pd.to_datetime(df_time['time']).dt.days

In [ ]:
sns.histplot(x='time', data=df_time, hue='state')